In [37]:
# 1. Connect out notebook with out Google Drive and navigate to the directory in which we saved the data file(s)

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/E2S/Account_Finance_Model

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/E2S/Account_Finance_Model


In [38]:
# 1. Load what we will need for data manipulation
import numpy as np
import pandas as pd
from datetime import datetime
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import re

Plan:

1. Visualize the structure
2. Implement
3. Validate
4. Submit


In [39]:
# Loading Dataset:

df = pd.read_csv('Copy of Account Finance Model - Worker Payments (edited)(2).csv')
df = df.astype('string')
df = df.iloc[1:,:]
df.head(20)


,Place,Stadium,Venue,Labor and Wage Laws,Poolable Tip Percentage,Notes,Staff,Stand Lead,Area Lead,Site Lead,CFA/Protein Runner,Volunteer,Beer Tent Staff,Beer Tent Stand Lead,Vending Lead,Cashier,Attendant/Suite Runner,Hawkers,Allocation Notes
1,LSU,LSU,Venue,Labor and Wage Laws,Poolable Tip Percentage,Notes,Staff,Stand Lead,Area Lead,Site Lead,<NA>,<NA>,<NA>,<NA>,<NA>,Notes,<NA>,<NA>,<NA>
2,LSU,Tiger Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,All stands at LSU are paid at the alcohol rate,Alcohol: $120 minimum Event base: $50 ...,Alcohol: $180 minimum Event base: $110 ...,$250 minimum Event base: $150 Tips: ...,$350 minimum Event base: $250 Tips: ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,LSU,Pete Maravich Assembly Center (PMAC),Basketball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $150,$65 minimum Event base: $25 Tips: ...,$85 minimum Event base: $45 Tips: ...,<NA>,Event base: $150,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LSU,Alex Box Stadium,Baseball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $100 minimum Event base: $55.00 ...,$120 minimum Event base: $75.00 Ti...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,LSU,Tiger Park,Softball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $85 minimum Event base: $40 T...,$105 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,NC State,NC State,Venue,Labor and Wage Laws,Poolable Tip Percentage,<NA>,Staff,Stand Lead,<NA>,Site Lead,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,NC State,Carter-Finley Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,15% The local pool is by location. Staff and...,<NA>,Alcohol: $100 minimum Event base: $21 ...,Alcohol: $150 minimum Event base: $71 ...,<NA>,Sum: $350 minimum Event base: $250 T...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,UNC,UNC,Venue,Labor and Wage Laws,Poolable Tip Percentage,<NA>,Staff,Stand Lead,Area Lead,Site Lead,CFA/Protein Runner,Volunteer,Beer Tent Staff,Beer Tent Stand Lead,<NA>,<NA>,<NA>,<NA>,<NA>
9,UNC,Kenan Memorial Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,Alcohol: $100 minimum Event base: $22 ...,Alcohol: $150 minimum Event base: $72 ...,Sum: $250 minimum Event base: $150 T...,Sum: $350 minimum Event base: $250 T...,$20/hr,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,UNC,Dean E. Smith Center,Basketball,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,$85 minimum Event base: $35 Tips: ...,$110 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [40]:
# Check the data types of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 1 to 69
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Place                    69 non-null     string
 1   Stadium                  69 non-null     string
 2   Venue                    69 non-null     string
 3   Labor and Wage Laws      67 non-null     string
 4   Poolable Tip Percentage  69 non-null     string
 5   Notes                    10 non-null     string
 6   Staff                    65 non-null     string
 7   Stand Lead               66 non-null     string
 8   Area Lead                32 non-null     string
 9   Site Lead                67 non-null     string
 10  CFA/Protein Runner       4 non-null      string
 11  Volunteer                2 non-null      string
 12  Beer Tent Staff          1 non-null      string
 13  Beer Tent Stand Lead     1 non-null      string
 14  Vending Lead             8 non-null      str

In [41]:
# Let's clean all "field" rows
df = df[df["Venue"] != 'Venue']
print(df.shape)
df.head()

(46, 19)


,Place,Stadium,Venue,Labor and Wage Laws,Poolable Tip Percentage,Notes,Staff,Stand Lead,Area Lead,Site Lead,CFA/Protein Runner,Volunteer,Beer Tent Staff,Beer Tent Stand Lead,Vending Lead,Cashier,Attendant/Suite Runner,Hawkers,Allocation Notes
2,LSU,Tiger Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,All stands at LSU are paid at the alcohol rate,Alcohol: $120 minimum Event base: $50 ...,Alcohol: $180 minimum Event base: $110 ...,$250 minimum Event base: $150 Tips: ...,$350 minimum Event base: $250 Tips: ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,LSU,Pete Maravich Assembly Center (PMAC),Basketball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $150,$65 minimum Event base: $25 Tips: ...,$85 minimum Event base: $45 Tips: ...,<NA>,Event base: $150,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LSU,Alex Box Stadium,Baseball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $100 minimum Event base: $55.00 ...,$120 minimum Event base: $75.00 Ti...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,LSU,Tiger Park,Softball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $85 minimum Event base: $40 T...,$105 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,NC State,Carter-Finley Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,15% The local pool is by location. Staff and...,<NA>,Alcohol: $100 minimum Event base: $21 ...,Alcohol: $150 minimum Event base: $71 ...,<NA>,Sum: $350 minimum Event base: $250 T...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [42]:
df.head(20)

,Place,Stadium,Venue,Labor and Wage Laws,Poolable Tip Percentage,Notes,Staff,Stand Lead,Area Lead,Site Lead,CFA/Protein Runner,Volunteer,Beer Tent Staff,Beer Tent Stand Lead,Vending Lead,Cashier,Attendant/Suite Runner,Hawkers,Allocation Notes
2,LSU,Tiger Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,All stands at LSU are paid at the alcohol rate,Alcohol: $120 minimum Event base: $50 ...,Alcohol: $180 minimum Event base: $110 ...,$250 minimum Event base: $150 Tips: ...,$350 minimum Event base: $250 Tips: ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,LSU,Pete Maravich Assembly Center (PMAC),Basketball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $150,$65 minimum Event base: $25 Tips: ...,$85 minimum Event base: $45 Tips: ...,<NA>,Event base: $150,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LSU,Alex Box Stadium,Baseball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $100 minimum Event base: $55.00 ...,$120 minimum Event base: $75.00 Ti...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,LSU,Tiger Park,Softball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $85 minimum Event base: $40 T...,$105 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,NC State,Carter-Finley Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,15% The local pool is by location. Staff and...,<NA>,Alcohol: $100 minimum Event base: $21 ...,Alcohol: $150 minimum Event base: $71 ...,<NA>,Sum: $350 minimum Event base: $250 T...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,UNC,Kenan Memorial Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,Alcohol: $100 minimum Event base: $22 ...,Alcohol: $150 minimum Event base: $72 ...,Sum: $250 minimum Event base: $150 T...,Sum: $350 minimum Event base: $250 T...,$20/hr,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,UNC,Dean E. Smith Center,Basketball,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,$85 minimum Event base: $35 Tips: ...,$110 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11,UNC,Boshamer Stadium,Baseball,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,$85 minimum Event base: $40 Tips: $...,$105 minimum Event base: $60 Tips:...,<NA>,Event base: $175 Double header: $262.50 Tourn...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
12,UNC,Carmichael Arena,Arena,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,$75 minimum Event base: $25 Tips: ...,$110 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13,UNC,Dorrance Field,Soccer Lacrosse,Non tipped work: $7.25/hour Tipped work: $2.13...,15%,<NA>,$75 minimum Event base: $25 Tips: ...,$110 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,$75 minimum Event base: $25 Tips: ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [20]:
# #TEST:
# # Let's attach parent suffix to the "Event base" and "Tips"
# inner_dict = {'Staff': [['Alcohol', '120'],
#   ['Event base', '50'],
#   ['Tips', '70'],
#   ['Food', '140'],
#   ['Event base', '70'],
#   ['Tips', '70']],
#   'Stand Lead ': [['Alcohol', '180'],
#   ['Event base', '110'],
#   ['Tips', '70'],
#   ['Food', '200'],
#   ['Event base', '130'],
#   ['Tips', '70']],
#   'Area Lead': [['Sum', '250'],
#   ['Event base', '150'],
#   ['Tips: match,', '100']],
#   'Site Lead ': [['Sum', '350'],
#   ['Event base', '250'],
#   ['Tips: match,', '100']]}

# column = 'Staff'

#   #  {'Staff': [['Sum', '65'],
#   #    ['Event base', '25'],
#   #    ['Tips', '40'],
#   #    ['NCAA Tourney', '95'],
#   #    ['Event base', '50'],
#   #    ['Tips', '45']],
#   #   'Stand Lead ': [['Sum', '85'],
#   #    ['Event base', '45'],
#   #    ['Tips', '40'],
#   #    ['NCAA Tourney', '130'],
#   #    ['Event base', '85'],
#   #    ['Tips', '45']]}
# for column in ["Staff", "Stand Lead ", "Area Lead", "Site Lead "]:
#   print("!!!!!!!!!!!!!!!!!!!!!!! Column: ", column)
#   for i in range(len(inner_dict[column])):
#     if i == 0:
#       parent_i = 0
#     if inner_dict[column][i][0] == "Event base":
#       inner_dict[column][i][0] = inner_dict[column][parent_i][0]+"_Event base"
#     elif inner_dict[column][i][0] == "Tips":
#       inner_dict[column][i][0] = inner_dict[column][parent_i][0]+"_Tips"
#     else:
#       parent_i = i
#     print("Parent: ", inner_dict[column][parent_i][0])
#     print("Current: ", inner_dict[column][i])
#   inner_dict

In [44]:
# Creating a dict:
out_dict = {}
columns_names = df.columns.values.tolist()
for id in range(len(columns_names)): # formating columns
  old_column_name = columns_names[id]
  columns_names[id] = columns_names[id].strip()
  df = df.rename(columns={f"{old_column_name}": f"{old_column_name.strip()}"})

print("Original DF columns: ", df.columns.values.tolist())
print("New DF columns: ", columns_names)

# Modify this line to create a copy of the list first
column_names_looping = columns_names[6:-1].copy()
column_names_looping.insert(0, str(columns_names[3])) # Then insert the element
print(columns_names[3])

for index, row in df.iterrows():
  print("Stadium: ", row["Stadium"])
  stadium = row["Stadium"].strip()
  inner_dict = {} # create inner_dict

  for column in column_names_looping:
    if pd.isna(row[column]):
      continue

    inner_dict[column] = row[column].split("\n")
    # print(" Stadium: ", stadium, " Column: ", column, "\nEND\n")

    for i in range(len(inner_dict[column])):
      item = inner_dict[column][i]
      if item == "":
        continue
      item = item.split("$")
      if item[0] == "":
        item[0] = "Sum"

      item[0] = item[0].strip(": ") # "-1" to account for ":" charecter
      item[1] = re.split('/| |/r', item[1])[0] # "/ worker" to account for "/worker"
      inner_dict[column][i] = item

      # print(item)

    # Let's attach parent suffix to the "Event base" and "Tips"
    parent_i = 0
    print(inner_dict[column])
    for i in range(len(inner_dict[column])):
      if (inner_dict[column] == None):
        break
      if (inner_dict[column][i] == ""):
        parent_i += 1
        continue
      print("Current: ", inner_dict[column][i])
      print("Parent: ", inner_dict[column][parent_i][0])
      print("Current Index: ", i)
      if inner_dict[column][i][0] == "Event base":
        inner_dict[column][i][0] = inner_dict[column][parent_i][0]+"_Event base"
      elif inner_dict[column][i][0] == "Tips":
        inner_dict[column][i][0] = inner_dict[column][parent_i][0]+"_Tips"
      else:
        parent_i = i
      print("Post Current: ", inner_dict[column][i])


  out_dict[stadium] = inner_dict # saving inner_dict into out_dict


Original DF columns:  ['Place', 'Stadium', 'Venue', 'Labor and Wage Laws', 'Poolable Tip Percentage', 'Notes', 'Staff', 'Stand Lead', 'Area Lead', 'Site Lead', 'CFA/Protein Runner', 'Volunteer', 'Beer Tent Staff', 'Beer Tent Stand Lead', 'Vending Lead', 'Cashier', 'Attendant/Suite Runner', 'Hawkers', 'Allocation Notes']
New DF columns:  ['Place', 'Stadium', 'Venue', 'Labor and Wage Laws', 'Poolable Tip Percentage', 'Notes', 'Staff', 'Stand Lead', 'Area Lead', 'Site Lead', 'CFA/Protein Runner', 'Volunteer', 'Beer Tent Staff', 'Beer Tent Stand Lead', 'Vending Lead', 'Cashier', 'Attendant/Suite Runner', 'Hawkers', 'Allocation Notes']
Labor and Wage Laws
Stadium:  Tiger Stadium
[['Non tipped work', '7.25'], ['Tipped work', '2.13']]
Current:  ['Non tipped work', '7.25']
Parent:  Non tipped work
Current Index:  0
Post Current:  ['Non tipped work', '7.25']
Current:  ['Tipped work', '2.13']
Parent:  Non tipped work
Current Index:  1
Post Current:  ['Tipped work', '2.13']
[['Alcohol', '120'], [

In [45]:
out_dict

{'Tiger Stadium': {'Labor and Wage Laws': [['Non tipped work', '7.25'],
   ['Tipped work', '2.13']],
  'Staff': [['Alcohol', '120'],
   ['Alcohol_Event base', '50'],
   ['Alcohol_Tips', '70'],
   ['Food', '140'],
   ['Food_Event base', '70'],
   ['Food_Tips', '70']],
  'Stand Lead': [['Alcohol', '180'],
   ['Alcohol_Event base', '110'],
   ['Alcohol_Tips', '70'],
   ['Food', '200'],
   ['Food_Event base', '130'],
   ['Food_Tips', '70']],
  'Area Lead': [['Sum', '250'],
   ['Sum_Event base', '150'],
   ['Sum_Tips', '100']],
  'Site Lead': [['Sum', '350'],
   ['Sum_Event base', '250'],
   ['Sum_Tips', '100']]},
 'Pete Maravich Assembly Center (PMAC)': {'Labor and Wage Laws': [['Non tipped work',
    '7.25'],
   ['Tipped work', '2.13']],
  'Staff': [['Sum', '65'],
   ['Sum_Event base', '25'],
   ['Sum_Tips', '40'],
   ['NCAA Tourney', '95'],
   ['NCAA Tourney_Event base', '50'],
   ['NCAA Tourney_Tips', '45']],
  'Stand Lead': [['Sum', '85'],
   ['Sum_Event base', '45'],
   ['Sum_Tips', '

In [46]:
# Flatten the dictionary
flattened_data = []
for outer_key, inner_dict in out_dict.items():
    for inner_key, values in inner_dict.items():
        for value in values:
            # Check if value is a list with at least two elements before accessing value[1]
            if isinstance(value, list) and len(value) >= 2:
                flattened_data.append((outer_key, f"{inner_key.strip()}_{value[0]}", value[1]))
            else:
                print(f"Skipping invalid value: {value}") # Print a message for debugging

# Create a DataFrame from the flattened data
df_cooking = pd.DataFrame(flattened_data, columns=['Stadium', 'Category', 'Amount'])

# Handle duplicate entries by concatenating them
df_grouped = df_cooking.groupby(['Stadium', 'Category'], as_index=False).agg(lambda x: ' | '.join(x))

# Pivot the DataFrame to get the desired format
df_pivot = df_grouped.pivot(index='Stadium', columns='Category', values='Amount')

# Display the resulting DataFrame
df_pivot.reset_index(inplace=True)
print(df_pivot)

Skipping invalid value: 
Skipping invalid value: 
Category                                    Stadium Area Lead_Sum  \
0                                  Alex Box Stadium           NaN   
1                                       All Alcohol           250   
2                       Alpine Valley Music Theatre           250   
3                                     Beckham Field           NaN   
4                                  Boshamer Stadium           NaN   
5                               Bridgeforth Stadium           250   
6                                  Carmichael Arena           NaN   
7                             Carter-Finley Stadium           NaN   
8                                  Cassell Coliseum           NaN   
9                                     Clark-LeClair           NaN   
10                              Colonial Life Arena           NaN   
11                                  DRV PNK Stadium           140   
12                        Dan Magill Tennis Complex  

In [51]:
df_pivot.shape

(45, 105)

In [52]:
df.shape

(46, 19)

In [55]:
df.head()

,Place,Stadium,Venue,Labor and Wage Laws,Poolable Tip Percentage,Notes,Staff,Stand Lead,Area Lead,Site Lead,CFA/Protein Runner,Volunteer,Beer Tent Staff,Beer Tent Stand Lead,Vending Lead,Cashier,Attendant/Suite Runner,Hawkers,Allocation Notes
2,LSU,Tiger Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,All stands at LSU are paid at the alcohol rate,Alcohol: $120 minimum Event base: $50 ...,Alcohol: $180 minimum Event base: $110 ...,$250 minimum Event base: $150 Tips: ...,$350 minimum Event base: $250 Tips: ...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,LSU,Pete Maravich Assembly Center (PMAC),Basketball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $150,$65 minimum Event base: $25 Tips: ...,$85 minimum Event base: $45 Tips: ...,<NA>,Event base: $150,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,LSU,Alex Box Stadium,Baseball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $100 minimum Event base: $55.00 ...,$120 minimum Event base: $75.00 Ti...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,LSU,Tiger Park,Softball,Non tipped work: $7.25/hour Tipped work: $2.13...,100%,Cecelia Shanks $200 approved by ML & MH,Sum: $85 minimum Event base: $40 T...,$105 minimum Event base: $60 Tips:...,<NA>,Event base: $175,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,NC State,Carter-Finley Stadium,Football,Non tipped work: $7.25/hour Tipped work: $2.13...,15% The local pool is by location. Staff and...,<NA>,Alcohol: $100 minimum Event base: $21 ...,Alcohol: $150 minimum Event base: $71 ...,<NA>,Sum: $350 minimum Event base: $250 T...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [56]:
# Let's join old df with df_pivot:
final_df = df.iloc[:, :6].merge(df_pivot, how= "left", on='Stadium')
final_df.drop(columns=['Labor and Wage Laws'], inplace=True)
print(final_df.shape)
final_df.head(30)

(46, 109)


,Place,Stadium,Venue,Poolable Tip Percentage,Notes,Area Lead_Sum,Area Lead_Sum_Event base,Area Lead_Sum_Tips,Attendant/Suite Runner_Sum,Attendant/Suite Runner_Sum_Event base,Attendant/Suite Runner_Sum_Tips,CFA/Protein Runner_Sum,CFA/Protein Runner_Sum_Event base,Cashier_Sum,Cashier_Sum_Event base,Cashier_Sum_Tips,Hawkers_Sum,Hawkers_Sum_Event base,Hawkers_Sum_Tips,Labor and Wage Laws_Non tipped work,Labor and Wage Laws_Tipped work,Site Lead_Double header,Site Lead_Event base_Event base,Site Lead_Event base_Event base_Tips,Site Lead_Event day,Site Lead_Non event day,Site Lead_Sum,Site Lead_Sum_Event base,Site Lead_Sum_Tips,Site Lead_Tournament,Staff_Alcohol,Staff_Alcohol (Visitors),Staff_Alcohol (Visitors)_Event base,Staff_Alcohol (Visitors)_Tips,Staff_Alcohol + hawking,Staff_Alcohol + hawking_Event base,Staff_Alcohol + hawking_Tips,Staff_Alcohol_Event base,Staff_Alcohol_Tips,Staff_Amphitheater Stands,Staff_Amphitheater Stands_Event base,Staff_Amphitheater Stands_Tips,Staff_Beer carts,Staff_Beer carts_Event base,Staff_Beer carts_Tips,Staff_Double header,Staff_Double header_Event base,Staff_Double header_Tips,Staff_Doubleheader,Staff_Doubleheader_Event base,Staff_Doubleheader_Tips,Staff_Food,Staff_Food_Event base,Staff_Food_Tips,Staff_Gamecock Park,Staff_Gamecock Park_Event base,Staff_Gamecock Park_Tips,Staff_Hawking,Staff_Hawking_Event base,Staff_Hawking_Tips,Staff_NCAA Tourney,Staff_NCAA Tourney_Event base,Staff_NCAA Tourney_Tips,Staff_Premium (Genome + Kenan Pines),Staff_Premium server,Staff_Premium server_Event base,Staff_Premium server_Tips,Staff_Stock Runners,Staff_Sum,Staff_Sum_Event base,Staff_Sum_Tips,Staff_Tournament,Staff_Tournament_Event base,Staff_Tournament_Tips,Staff_ULine East + Generac 5,Staff_ULine East + Generac 5_Event base,Staff_ULine East + Generac 5_Tips,Stand Lead_Alcohol,Stand Lead_Alcohol_Event base,Stand Lead_Alcohol_Tips,Stand Lead_Double header,Stand Lead_Double header_Event base,Stand Lead_Double header_Tips,Stand Lead_Doubleheader,Stand Lead_Doubleheader_Event base,Stand Lead_Doubleheader_Tips,Stand Lead_Event base_Event base,Stand Lead_Event base_Event base_Tips,Stand Lead_Food,Stand Lead_Food_Event base,Stand Lead_Food_Tips,Stand Lead_Hawking,Stand Lead_Hawking_Event base,Stand Lead_Hawking_Tips,Stand Lead_NCAA Tourney,Stand Lead_NCAA Tourney_Event base,Stand Lead_NCAA Tourney_Tips,Stand Lead_Sum,Stand Lead_Sum_Event base,Stand Lead_Sum_Tips,Stand Lead_Tournament,Stand Lead_Tournament_Event base,Stand Lead_Tournament_Tips,Vending Lead_Sum,Vending Lead_Sum_Event base,Vending Lead_Sum_Tips,Volunteer_Sum,Volunteer_Sum_Event base,Volunteer_Sum_Tips
0,LSU,Tiger Stadium,Football,100%,All stands at LSU are paid at the alcohol rate,250,150,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.25,2.13,NaN,NaN,NaN,NaN,NaN,350,250,100,NaN,120,NaN,NaN,NaN,NaN,NaN,NaN,50,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140,70,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180,110,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,130,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LSU,Pete Maravich Assembly Center (PMAC),Basketball,100%,Cecelia Shanks $150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.25,2.13,NaN,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,50,45,NaN,NaN,NaN,NaN,NaN,65,25,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130,85,45,85,45,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LSU,Alex Box Stadium,Baseball,100%,Cecelia Shanks $200 approved by ML & MH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.25,2.13,NaN,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,55.00,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [57]:
# prompt: how to place "Labor and Wage Laws_Tipped work" and "Labor and Wage Laws_Non tipped work" columns to be at position 5 and 6 instead

# Get the columns you want to move
col1 = final_df.pop("Labor and Wage Laws_Tipped work")
col2 = final_df.pop("Labor and Wage Laws_Non tipped work")

# Insert them at the desired positions
final_df.insert(5, "Labor and Wage Laws_Tipped work", col1)
final_df.insert(6, "Labor and Wage Laws_Non tipped work", col2)

print(final_df.shape)
final_df.head(30)


<ipython-input-57-8427a210911e>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df.insert(5, "Labor and Wage Laws_Tipped work", col1)
<ipython-input-57-8427a210911e>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df.insert(6, "Labor and Wage Laws_Non tipped work", col2)


(46, 109)


,Place,Stadium,Venue,Poolable Tip Percentage,Notes,Labor and Wage Laws_Tipped work,Labor and Wage Laws_Non tipped work,Area Lead_Sum,Area Lead_Sum_Event base,Area Lead_Sum_Tips,Attendant/Suite Runner_Sum,Attendant/Suite Runner_Sum_Event base,Attendant/Suite Runner_Sum_Tips,CFA/Protein Runner_Sum,CFA/Protein Runner_Sum_Event base,Cashier_Sum,Cashier_Sum_Event base,Cashier_Sum_Tips,Hawkers_Sum,Hawkers_Sum_Event base,Hawkers_Sum_Tips,Site Lead_Double header,Site Lead_Event base_Event base,Site Lead_Event base_Event base_Tips,Site Lead_Event day,Site Lead_Non event day,Site Lead_Sum,Site Lead_Sum_Event base,Site Lead_Sum_Tips,Site Lead_Tournament,Staff_Alcohol,Staff_Alcohol (Visitors),Staff_Alcohol (Visitors)_Event base,Staff_Alcohol (Visitors)_Tips,Staff_Alcohol + hawking,Staff_Alcohol + hawking_Event base,Staff_Alcohol + hawking_Tips,Staff_Alcohol_Event base,Staff_Alcohol_Tips,Staff_Amphitheater Stands,Staff_Amphitheater Stands_Event base,Staff_Amphitheater Stands_Tips,Staff_Beer carts,Staff_Beer carts_Event base,Staff_Beer carts_Tips,Staff_Double header,Staff_Double header_Event base,Staff_Double header_Tips,Staff_Doubleheader,Staff_Doubleheader_Event base,Staff_Doubleheader_Tips,Staff_Food,Staff_Food_Event base,Staff_Food_Tips,Staff_Gamecock Park,Staff_Gamecock Park_Event base,Staff_Gamecock Park_Tips,Staff_Hawking,Staff_Hawking_Event base,Staff_Hawking_Tips,Staff_NCAA Tourney,Staff_NCAA Tourney_Event base,Staff_NCAA Tourney_Tips,Staff_Premium (Genome + Kenan Pines),Staff_Premium server,Staff_Premium server_Event base,Staff_Premium server_Tips,Staff_Stock Runners,Staff_Sum,Staff_Sum_Event base,Staff_Sum_Tips,Staff_Tournament,Staff_Tournament_Event base,Staff_Tournament_Tips,Staff_ULine East + Generac 5,Staff_ULine East + Generac 5_Event base,Staff_ULine East + Generac 5_Tips,Stand Lead_Alcohol,Stand Lead_Alcohol_Event base,Stand Lead_Alcohol_Tips,Stand Lead_Double header,Stand Lead_Double header_Event base,Stand Lead_Double header_Tips,Stand Lead_Doubleheader,Stand Lead_Doubleheader_Event base,Stand Lead_Doubleheader_Tips,Stand Lead_Event base_Event base,Stand Lead_Event base_Event base_Tips,Stand Lead_Food,Stand Lead_Food_Event base,Stand Lead_Food_Tips,Stand Lead_Hawking,Stand Lead_Hawking_Event base,Stand Lead_Hawking_Tips,Stand Lead_NCAA Tourney,Stand Lead_NCAA Tourney_Event base,Stand Lead_NCAA Tourney_Tips,Stand Lead_Sum,Stand Lead_Sum_Event base,Stand Lead_Sum_Tips,Stand Lead_Tournament,Stand Lead_Tournament_Event base,Stand Lead_Tournament_Tips,Vending Lead_Sum,Vending Lead_Sum_Event base,Vending Lead_Sum_Tips,Volunteer_Sum,Volunteer_Sum_Event base,Volunteer_Sum_Tips
0,LSU,Tiger Stadium,Football,100%,All stands at LSU are paid at the alcohol rate,2.13,7.25,250,150,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350,250,100,NaN,120,NaN,NaN,NaN,NaN,NaN,NaN,50,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140,70,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180,110,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,130,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LSU,Pete Maravich Assembly Center (PMAC),Basketball,100%,Cecelia Shanks $150,2.13,7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,50,45,NaN,NaN,NaN,NaN,NaN,65,25,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130,85,45,85,45,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LSU,Alex Box Stadium,Baseball,100%,Cecelia Shanks $200 approved by ML & MH,2.13,7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,55.00,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [58]:
# save as csv file:
df_pivot.to_csv('flattened_data.csv', index=False)

In [59]:
# save as csv file:
final_df.to_csv('Account Finance Model - Machine Readable Format.csv', index=False)